# Attribute Information:

## Input variables:
### bank client data:
1 - age (numeric)  
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')  
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)  
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')  
5 - default: has credit in default? (categorical: 'no','yes','unknown')  
6 - housing: has housing loan? (categorical: 'no','yes','unknown')  
7 - loan: has personal loan? (categorical: 'no','yes','unknown')  
### related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')  
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')  
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')  
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.  
### other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)  
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)  
14 - previous: number of contacts performed before this campaign and for this client (numeric)  
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')  
### social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)  
17 - cons.price.idx: consumer price index - monthly indicator (numeric)  
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)  
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)  
20 - nr.employed: number of employees - quarterly indicator (numeric)  

### Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')  

In [1]:
import numpy as np
import pandas as pd

In [2]:
df= pd.read_csv('bank-full.csv', sep=';')

In [3]:
# check for nulls
df.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

## investgate the diffrence between unknown and other in poutcomes column

In [4]:
df[df['poutcome']=='unknown']['pdays'].value_counts()

-1      36954
 416        1
 98         1
 168        1
 528        1
 188        1
Name: pdays, dtype: int64

In [5]:
df[df['poutcome']=='other']['pdays'].value_counts()

2      35
8      23
364    21
343    21
182    19
       ..
106     1
73      1
71      1
69      1
838     1
Name: pdays, Length: 386, dtype: int64

The 'unknown' tag is mostly related to clients who have not been in any previous campaigns, while the 'other' is relateed to clints where the campaign didn't work and didn't fail.

# convert object data

In [32]:
# create dummies and merge them with df
a= pd.get_dummies(df[['job', 'marital', 'education', 'poutcome', 'contact']], drop_first=True)
df= pd.concat([df,a], axis=1)
df.drop(['job','marital', 'education', 'poutcome', 'contact'], axis=1, inplace=True)

In [39]:
# convert 'yes', 'no' to 1 and 0
for i in df.select_dtypes('object').columns[:-2]:
    df[i]= df[i].apply(lambda x : 1 if x=='yes' else 0)

In [46]:
# group every 3 months into one group. 
df['month']= df['month'].apply(lambda x : 1 if x in ['jan', 'feb', 'mar'] else (2 if x in ['apr', 'may', 'jun'] else (3 if x in ['jul', 'aug','sep'] else 4) ) )

In [49]:
# save new df
df.to_csv('bank-clean.csv')